# Prepareren dummy data

In [0]:
%run /Repos/r.j.breeuwer@amsterdam.nl/doorontwikkeling_historisering/src/dpms/doorontwikkeling_hist

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col
import random
from datetime import datetime
import sys
from pyspark.sql.functions import *
from pyspark.sql import Row, DataFrame
from pyspark.sql.window import Window

aantal_records = 200
n_verwijder = int(0.05*aantal_records)
n_reinsert = int(n_verwijder * 0.5)

In [0]:
# Maak een SparkSession aan
spark = SparkSession.builder.appName("voorbeeld").getOrCreate()

# Definieer het schema
schema_5 = StructType([
    StructField("account_id", IntegerType(), nullable=True),
    StructField("account_naam", StringType(), nullable=True),
    StructField("aantal_sporten", IntegerType(), nullable=True),
    StructField("type_sport", StringType(), nullable=True),
    StructField("stadsdeel", StringType(), nullable=True),
    StructField("gewijzigd", BooleanType(), nullable=False)  # Voeg een extra kolom "gewijzigd" toe
])

# Lijst met 50 sporten
sports_list = ["Voetbal", "Basketbal", "Tennis", "Volleybal", "Zwemmen", "Fietsen", "Hardlopen", "Golf", "Honkbal",
               "Hockey", "Badminton", "Tafeltennis", "Skiën", "Snowboarden", "Boksen", "Vechtsporten", "Cricket", "Rugby",
               "Softbal", "Wandelen", "Gewichtheffen", "Klimmen", "Surfen", "Skateboarden", "Yoga", "Pilates", "Dansen", "Schermen", "Gymnastiek",
               "Atletiek", "Roeien", "Kanoën", "Schaken", "Schaatsen", "Zeilen", "Mountainbiken", "Korfbal", "Turnen", "Handbal", "Squash",
               "Boogschieten", "Paardrijden", "Gymnastiek", "Triatlon", "Rolschaatsen", "Zeilen", "Judo", "Wielrennen", "Jiu-Jitsu"]

# Lijst met stadsdelen
stadsdelen = ["Noord", "Oost", "Zuid", "West"]

# Functie om een willekeurige record te genereren
def genereer_willekeurige_record(account_id):
    account_naam = f"Account_{account_id}"
    aantal_sporten = random.randint(1, 25)
    type_sport = random.sample(sports_list, aantal_sporten)
    stadsdeel = random.choice(stadsdelen)
    
    return (account_id, account_naam, aantal_sporten, type_sport, stadsdeel, False)  # Voeg False toe voor de kolom "gewijzigd"

# Genereer een records
records = [genereer_willekeurige_record(i) for i in range(1, aantal_records + 1)]

# Maak een DataFrame aan
df = spark.createDataFrame(records, schema=schema_5)

# Toon het DataFrame met de nieuwe kolom
df.display()

# Maak een tempview aan om initialiseren_df te testen
df.createOrReplaceTempView("test_dwh_historisering_temp_view")

account_id,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
3,Account_3,21,"[Triatlon, Handbal, Wandelen, Jiu-Jitsu, Klimmen, Schermen, Wielrennen, Zwemmen, Honkbal, Gymnastiek, Schaatsen, Yoga, Fietsen, Rugby, Skiën, Hockey, Atletiek, Gewichtheffen, Boogschieten, Paardrijden, Pilates]",West,false
4,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
8,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
9,Account_9,13,"[Schaken, Mountainbiken, Cricket, Rolschaatsen, Turnen, Skateboarden, Fietsen, Wielrennen, Jiu-Jitsu, Judo, Klimmen, Handbal, Zwemmen]",Zuid,false
10,Account_10,2,"[Wandelen, Schaatsen]",West,false


In [0]:
# Kijken of de aanpassing correct verwerkt worden!
# SQL-variant
# toepassen_historisering(nieuw_df="test_dwh_historisering_temp_view", 
#                         schema_catalog="dpms_dev.silver",
#                         rij_id_var="account_id")

# Python variant
toepassen_historisering(nieuw_df = df, 
                        schema_catalog="dpms_dev.silver",
                        business_key="account_id",
                        naam_nieuw_df="test_dwh_historisering")

Dit is de eerste keer dat je de tabel: test_dwh_historisering wilt historiseren. Historisering wordt nu toegepast...
Ontbrekende kolommen: geldig_van, geldig_tot, surrogaat_sleutel, record_actief, actie. Deze worden aan het DataFrame toegevoegd...
De tabel wordt nu opgeslagen in dpms_dev.silver onder de naam: test_dwh_historisering


'Historisering is toegepast!'

In [0]:
########### AANGEPAST en VERWIJDERT ####################
# Lijst met 50 sporten
sports_list = ["Voetbal", "Basketbal", "Tennis", "Volleybal", "Zwemmen", "Fietsen", "Hardlopen", "Golf", "Honkbal",
               "Hockey", "Badminton", "Tafeltennis", "Skiën", "Snowboarden", "Boksen", "Vechtsporten", "Cricket", "Rugby",
               "Softbal", "Wandelen", "Gewichtheffen", "Klimmen", "Surfen", "Skateboarden", "Yoga", "Pilates", "Dansen", "Schermen", "Gymnastiek",
               "Atletiek", "Roeien", "Kanoën", "Schaken", "Schaatsen", "Zeilen", "Mountainbiken", "Korfbal", "Turnen", "Handbal", "Squash",
               "Boogschieten", "Paardrijden", "Gymnastiek", "Triatlon", "Rolschaatsen", "Zeilen", "Judo", "Wielrennen", "Jiu-Jitsu"]

# Lijst met stadsdelen
stadsdelen = ["Noord", "Oost", "Zuid", "West"]

# Pas 5% van de records aan en maak een tweede DataFrame aan
df_dwh = spark.table('dpms_dev.silver.test_dwh_historisering')

# Voeg een kolom 'random_value' toe met willekeurige waarden tussen 0 en 1
df_aangepast = df_dwh.withColumn("random_value", rand())

# Pas ongeveer 5% van de records aan op basis van de 'random_value'
df_aangepast = (
    df_aangepast
    .withColumn("gewijzigd", when(col("random_value") <= 0.05, True).otherwise(col("gewijzigd")))
    .withColumn("stadsdeel", when(col("random_value") <= 0.05, random.choice(stadsdelen)).otherwise(col("stadsdeel")))
    .withColumn("aantal_sporten", when(col("random_value") <= 0.05, col("aantal_sporten") + 1).otherwise(col("aantal_sporten")))
    .drop("random_value")  # Verwijder de tijdelijke kolom
    #.filter(col("gewijzigd") == True)  # Hier de vergelijking met True toegevoegd
)


#Filter 5% records eruit om de actie:delete functionaliteit te ontwikkelen.
unique_ids_set = set(df_aangepast.select("account_id").rdd.flatMap(lambda x: x).collect())
random_numbers = random.sample(unique_ids_set, n_verwijder)

# Genereer een DataFrame met de willekeurige getallen
random_numbers_df = spark.createDataFrame([(num,) for num in random_numbers], ["account_id"])

# Voer een left anti join uit om rijen te filteren waar account_id overeenkomt met random_numbers
#df_aangepast_verwijdert = (df_aangepast.filter(~col("account_id").isin(random_numbers))) #isin is sloom
df_aangepast_verwijderd = df_aangepast.join(
    random_numbers_df, 
    on="account_id", 
    how="left_anti"
)

df_aangepast_verwijderd.display()

/root/.ipykernel/4049/command-1205613352114518-60370643:31: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random_numbers = random.sample(unique_ids_set, n_verwijder)


account_id,record_actief,surrogaat_sleutel,geldig_van,geldig_tot,actie,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,true,1,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,true,2,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
4,true,4,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,true,5,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,true,6,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,true,7,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
9,true,9,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_9,13,"[Schaken, Mountainbiken, Cricket, Rolschaatsen, Turnen, Skateboarden, Fietsen, Wielrennen, Jiu-Jitsu, Judo, Klimmen, Handbal, Zwemmen]",Zuid,false
10,true,10,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_10,2,"[Wandelen, Schaatsen]",West,false
11,true,11,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_11,19,"[Basketbal, Paardrijden, Volleybal, Triatlon, Gymnastiek, Skiën, Golf, Fietsen, Softbal, Squash, Yoga, Rolschaatsen, Zeilen, Voetbal, Zwemmen, Hardlopen, Dansen, Schermen, Boogschieten]",Oost,false
12,true,12,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_12,10,"[Zwemmen, Rolschaatsen, Triatlon, Skateboarden, Volleybal, Surfen, Jiu-Jitsu, Cricket, Gymnastiek, Schermen]",Oost,false


In [0]:
# Sla de standaard tabel op in silver
df_aangepast_verwijderd.write.saveAsTable(f'dpms_dev.silver.aangepast_test_dwh_historisering', mode='overwrite')

In [0]:
# SQL query to create a temporary view
sql_query = "SELECT * FROM dpms_dev.silver.aangepast_test_dwh_historisering"

# Create a DataFrame from the SQL query
pyspark_df = spark.sql(sql_query)

# Show the result DataFrame
pyspark_df.display()

account_id,record_actief,surrogaat_sleutel,geldig_van,geldig_tot,actie,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,true,1,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,true,2,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
4,true,4,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,true,5,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,true,6,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,true,7,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
9,true,9,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_9,13,"[Schaken, Mountainbiken, Cricket, Rolschaatsen, Turnen, Skateboarden, Fietsen, Wielrennen, Jiu-Jitsu, Judo, Klimmen, Handbal, Zwemmen]",Zuid,false
10,true,10,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_10,2,"[Wandelen, Schaatsen]",West,false
11,true,11,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_11,19,"[Basketbal, Paardrijden, Volleybal, Triatlon, Gymnastiek, Skiën, Golf, Fietsen, Softbal, Squash, Yoga, Rolschaatsen, Zeilen, Voetbal, Zwemmen, Hardlopen, Dansen, Schermen, Boogschieten]",Oost,false
12,true,12,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_12,10,"[Zwemmen, Rolschaatsen, Triatlon, Skateboarden, Volleybal, Surfen, Jiu-Jitsu, Cricket, Gymnastiek, Schermen]",Oost,false


In [0]:
%run /Repos/r.j.breeuwer@amsterdam.nl/doorontwikkeling_historisering/src/dpms/doorontwikkeling_hist

In [0]:
# Kijken of de aanpassing correct verwerkt worden!
# SQL-variant
# toepassen_historisering(nieuw_df="test_dwh_historisering_temp_view", 
#                         schema_catalog="dpms_dev.silver",
#                         rij_id_var="account_id")

# Python variant
toepassen_historisering(nieuw_df = pyspark_df, 
                        schema_catalog="dpms_dev.silver",
                        business_key="account_id",
                        naam_nieuw_df="test_dwh_historisering")

De tabel: test_dwh_historisering bevindt zich in de Unity Catalogus onder het volgende schema: dpms_dev.silver
{31, 116, 55, 25, 140, 125, 95}
Verwijderd: {64, 3, 196, 198, 40, 8, 188, 88, 92, 191}
set()
set()


account_id,record_actief,surrogaat_sleutel,geldig_van,geldig_tot,actie,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,true,1,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,true,2,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
3,false,3,2024-01-05T11:02:37Z,2024-01-05T11:07:34Z,deleted,Account_3,21,"[Triatlon, Handbal, Wandelen, Jiu-Jitsu, Klimmen, Schermen, Wielrennen, Zwemmen, Honkbal, Gymnastiek, Schaatsen, Yoga, Fietsen, Rugby, Skiën, Hockey, Atletiek, Gewichtheffen, Boogschieten, Paardrijden, Pilates]",West,false
4,true,4,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,true,5,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,true,6,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,true,7,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
8,false,8,2024-01-05T11:02:37Z,2024-01-05T11:07:34Z,deleted,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
9,true,9,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_9,13,"[Schaken, Mountainbiken, Cricket, Rolschaatsen, Turnen, Skateboarden, Fietsen, Wielrennen, Jiu-Jitsu, Judo, Klimmen, Handbal, Zwemmen]",Zuid,false
10,true,10,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_10,2,"[Wandelen, Schaatsen]",West,false


De tabel wordt nu opgeslagen in dpms_dev.silver onder de naam: test_dwh_historisering


'Historisering is toegepast!'

# Extra data

In [0]:
%sql
CLEAR CACHE

In [0]:
########### EXTRA ####################

# Pas 5% van de records aan en maak een tweede DataFrame aan
df_dwh = spark.table('dpms_dev.silver.test_dwh_historisering')

# Genereer 10% extra records met een nieuwe teller die verder gaat dan het aantal_records
x_r = int(aantal_records * 1.1)
extra_records = [genereer_willekeurige_record(i) for i in range(aantal_records + 1, x_r)]

# Maak een DataFrame aan voor de extra records
df_extra = spark.createDataFrame(extra_records, schema=schema_5)
df_extra = voeg_aangepaste_surrogaatsleutel_toe(df_extra)

id_reinserten = random.sample(random_numbers,n_reinsert)

# Pas de account_id's van n_rijen aan om reinsert te testen
df_extra_2 = (df_extra
        .withColumn("account_id",
                    when(row_number().over(Window.orderBy("surrogaat_sleutel")) <= n_reinsert,
                    array(*[lit(value) for value in id_reinserten])[row_number().over(Window.orderBy("surrogaat_sleutel"))])
        .otherwise(col("account_id")))
        .filter(col("account_id").isNotNull())
        .limit( (n_reinsert-1))
)

uitzondering_kolom = ["geldig_van", "geldig_tot", "surrogaat_sleutel", "record_actief", "actie"]
df_dwh_temp = df_dwh.drop(*uitzondering_kolom)

df_extra_3 = (df_extra.union(df_extra_2).drop("surrogaat_sleutel"))
df_extra_4 = (df_dwh_temp.union(df_extra_3))
df_extra_4.display()

account_id,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
3,Account_3,21,"[Triatlon, Handbal, Wandelen, Jiu-Jitsu, Klimmen, Schermen, Wielrennen, Zwemmen, Honkbal, Gymnastiek, Schaatsen, Yoga, Fietsen, Rugby, Skiën, Hockey, Atletiek, Gewichtheffen, Boogschieten, Paardrijden, Pilates]",West,false
4,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
8,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
9,Account_9,13,"[Schaken, Mountainbiken, Cricket, Rolschaatsen, Turnen, Skateboarden, Fietsen, Wielrennen, Jiu-Jitsu, Judo, Klimmen, Handbal, Zwemmen]",Zuid,false
10,Account_10,2,"[Wandelen, Schaatsen]",West,false


In [0]:
# Sla extra toevoeging op in silver
df_extra_4.write.saveAsTable(f'dpms_dev.silver.extra_test_dwh_historisering', mode='overwrite')

In [0]:
%run /Repos/r.j.breeuwer@amsterdam.nl/doorontwikkeling_historisering/src/dpms/doorontwikkeling_hist

# Aanpassing


In [0]:
%sql
-- vergeet niet het legen van de cache
CLEAR CACHE;

In [0]:
# SQL query to create a temporary view
sql_query = "SELECT * FROM dpms_dev.silver.extra_test_dwh_historisering"

# Create a DataFrame from the SQL query
extra_data = spark.sql(sql_query)

# Show the result DataFrame
extra_data.display()

account_id,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
3,Account_3,21,"[Triatlon, Handbal, Wandelen, Jiu-Jitsu, Klimmen, Schermen, Wielrennen, Zwemmen, Honkbal, Gymnastiek, Schaatsen, Yoga, Fietsen, Rugby, Skiën, Hockey, Atletiek, Gewichtheffen, Boogschieten, Paardrijden, Pilates]",West,false
4,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
8,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
9,Account_9,13,"[Schaken, Mountainbiken, Cricket, Rolschaatsen, Turnen, Skateboarden, Fietsen, Wielrennen, Jiu-Jitsu, Judo, Klimmen, Handbal, Zwemmen]",Zuid,false
10,Account_10,2,"[Wandelen, Schaatsen]",West,false


In [0]:
%run /Repos/r.j.breeuwer@amsterdam.nl/doorontwikkeling_historisering/src/dpms/doorontwikkeling_hist

In [0]:
# Nu testen of reinserted werkt!
# Python variant
toepassen_historisering(nieuw_df = extra_data, 
                        schema_catalog="dpms_dev.silver",
                        business_key="account_id",
                        naam_nieuw_df="test_dwh_historisering")

De tabel: test_dwh_historisering bevindt zich in de Unity Catalogus onder het volgende schema: dpms_dev.silver
set()
Verwijderd: set()
{40, 8, 188, 191}
{201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219}


account_id,record_actief,surrogaat_sleutel,geldig_van,geldig_tot,actie,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,true,1,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,true,2,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
3,false,3,2024-01-05T11:02:37Z,2024-01-05T11:07:34Z,deleted,Account_3,21,"[Triatlon, Handbal, Wandelen, Jiu-Jitsu, Klimmen, Schermen, Wielrennen, Zwemmen, Honkbal, Gymnastiek, Schaatsen, Yoga, Fietsen, Rugby, Skiën, Hockey, Atletiek, Gewichtheffen, Boogschieten, Paardrijden, Pilates]",West,false
4,true,4,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,true,5,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,true,6,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,true,7,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
8,true,208,2024-01-05T11:14:08Z,9999-12-31T23:59:59Z,reinserted,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
8,false,8,2024-01-05T11:02:37Z,2024-01-05T11:14:08Z,deleted,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
8,true,225,2024-01-05T11:14:08Z,9999-12-31T23:59:59Z,reinserted,Account_203,10,"[Volleybal, Mountainbiken, Schermen, Korfbal, Turnen, Boogschieten, Judo, Yoga, Klimmen, Vechtsporten]",Oost,false


De tabel wordt nu opgeslagen in dpms_dev.silver onder de naam: test_dwh_historisering


'Historisering is toegepast!'

In [0]:
%sql
CLEAR CACHE

In [0]:
%sql 
SELECT *
FROM dpms_dev.silver.test_dwh_historisering

account_id,record_actief,surrogaat_sleutel,geldig_van,geldig_tot,actie,account_naam,aantal_sporten,type_sport,stadsdeel,gewijzigd
1,true,1,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_1,24,"[Schermen, Tafeltennis, Dansen, Skiën, Kanoën, Hardlopen, Surfen, Handbal, Honkbal, Turnen, Jiu-Jitsu, Schaatsen, Gymnastiek, Schaken, Zwemmen, Gymnastiek, Snowboarden, Atletiek, Klimmen, Roeien, Golf, Fietsen, Hockey, Vechtsporten]",Noord,false
2,true,2,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_2,2,"[Gewichtheffen, Rugby]",Zuid,false
3,false,3,2024-01-05T11:02:37Z,2024-01-05T11:07:34Z,deleted,Account_3,21,"[Triatlon, Handbal, Wandelen, Jiu-Jitsu, Klimmen, Schermen, Wielrennen, Zwemmen, Honkbal, Gymnastiek, Schaatsen, Yoga, Fietsen, Rugby, Skiën, Hockey, Atletiek, Gewichtheffen, Boogschieten, Paardrijden, Pilates]",West,false
4,true,4,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_4,9,"[Honkbal, Roeien, Zeilen, Zwemmen, Paardrijden, Rugby, Klimmen, Wielrennen, Squash]",Oost,false
5,true,5,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_5,18,"[Triatlon, Zeilen, Voetbal, Mountainbiken, Handbal, Wandelen, Zeilen, Wielrennen, Boogschieten, Klimmen, Golf, Tennis, Skiën, Yoga, Softbal, Volleybal, Schermen, Boksen]",West,false
6,true,6,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_6,12,"[Jiu-Jitsu, Korfbal, Yoga, Surfen, Zeilen, Snowboarden, Skateboarden, Handbal, Atletiek, Schermen, Softbal, Dansen]",Oost,false
7,true,7,2024-01-05T11:02:37Z,9999-12-31T23:59:59Z,inserted,Account_7,13,"[Cricket, Schermen, Badminton, Wandelen, Squash, Snowboarden, Schaatsen, Basketbal, Turnen, Judo, Fietsen, Surfen, Atletiek]",Zuid,false
8,false,8,2024-01-05T11:02:37Z,2024-01-05T11:14:08Z,deleted,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
8,true,208,2024-01-05T11:14:08Z,9999-12-31T23:59:59Z,reinserted,Account_8,8,"[Tafeltennis, Hardlopen, Turnen, Handbal, Jiu-Jitsu, Korfbal, Tennis, Skateboarden]",Noord,false
8,true,225,2024-01-05T11:14:08Z,9999-12-31T23:59:59Z,reinserted,Account_203,10,"[Volleybal, Mountainbiken, Schermen, Korfbal, Turnen, Boogschieten, Judo, Yoga, Klimmen, Vechtsporten]",Oost,false
